<a href="https://colab.research.google.com/github/Flychuban/Space-Soup/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.2 MB/s eta 0:00:00


In [2]:
import requests
from transformers import pipeline
from bs4 import BeautifulSoup
import re

In [3]:
# Load summarization model from HuggingFace
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Get text from web page

In [4]:
def search_urls_space_news():
  #URL = "https://www.google.com/search?q=nasa+news&tbm=nws"
  URL = "https://www.google.com/search?sca_esv=568425080&rlz=1C1CHZN_enBG975BG975&sxsrf=AM9HkKmWvQEyiFIWj11sb6Ld6OCoRQ9Kaw:1695711800085&q=space.com+latest+articles&tbm=nws&source=lnms&sa=X&ved=2ahUKEwi_y_Db2seBAxXkRvEDHZ0BDmIQ0pQJegQIDBAB&biw=1536&bih=747&dpr=1.25"
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all(['a'])
  hrefs = [link['href'] for link in atags]
  return hrefs


In [5]:
hrefs = search_urls_space_news()

In [6]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [7]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [8]:
cleaned_urls = strip_unwanted_urls(hrefs, exclude_list)
cleaned_urls

['https://www.biospace.com/article/releases/galmed-announces-allowance-of-new-patent-for-aramchol-for-the-treatment-of-pulmonary-and-dermal-fibrosis/',
 'https://www.space.com/autumnal-equinox-fall-begins-september-2023',
 'https://www.space.com/nasa-may-have-unknowingly-found-and-killed-alien-life-on-mars-50-years-ago-scientist-claims',
 'https://www.space.com/spacex-crew-6-astronauts-international-space-station-farewell',
 'https://www.space.com/comet-nishimura-green-comet-expert-astronomer-explains',
 'https://www.space.com/japan-slim-moon-lander-mission-launch-preview',
 'https://www.space.com/nasa-new-horizons-budget-uncertainty',
 'https://www.space.com/european-space-agency-freedom-of-information',
 'https://www.google.com/search?sca_esv%3D568425080%26rlz%3D1C1CHZN_enBG975BG975%26sxsrf%3DAM9HkKmWvQEyiFIWj11sb6Ld6OCoRQ9Kaw:1695711800085%26q%3Dspace.com%2Blatest%2Barticles%26tbm%3Dnws%26source%3Dlnms%26sa%3DX%26ved%3D2ahUKEwi_y_Db2seBAxXkRvEDHZ0BDmIQ0pQJegQIDBAB%26biw%3D1536%26bih

In [9]:
def get_text_from_urls(urls):
  articles = []
  titles = []
  for url in urls:
    headers = {'User-Agent': 'Mozilla/5.0'}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    r = requests.get(url, headers=headers, cookies=cookies)
    if r.status_code != 200:
      continue

    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = [paragraph.text for paragraph in paragraphs]
    # words = ' '.join(text)
    article = ' '.join(text)
    article = article.rstrip()

    # Minimal number of symbols in article, prevent returning cookies or terms error
    if len(article) < 200:
      continue

    articles.append(article)
    url_h1_tags = soup.find_all(['h1'])
    url_titles = [title.text.strip() for title in url_h1_tags]
    titles.append(url_titles)

  return titles, articles

In [10]:
titles, articles = get_text_from_urls(cleaned_urls)

In [11]:
articles[0]

'Skip to main content \nPublished: Sep 26, 2023\n\n TEL AVIV, Israel, Sept. 26, 2023 /PRNewswire/ --\xa0Galmed Pharmaceuticals Ltd. (Nasdaq: GLMD) ("Galmed" or the "Company"), a clinical-stage biopharmaceutical company for liver, metabolic and fibrotic diseases announced today the allowance of a Japanese patent related to treatment of pulmonary and dermal fibrosis. A similar patent was already granted in Mexico. The approval of the patent in the US and the rest of the world is pending. With this latest patent, Galmed is strengthening and extending the patent protection of its lead compound, Aramchol, until November 2037. \xa0 \xa0 Previously, Galmed reported results showing significant anti-fibrotic effects of Aramchol in a pre-clinical model of lung fibrosis. Treatment with Aramchol resulted in statistically significant fibrosis improvement in a validated bleomycin model of lung fibrosis (IPF), comparable to Pirfenidone which is the gold standard treatment. Findings were seen across a

Split text from the article into chunks

In [12]:
sentences = []
for i, article in enumerate(articles):
  article = article.replace('.', '.<eos>')
  article = article.replace('!', '!<eos>')
  article = article.replace('?', '?<eos>')
  sentences.append(article.split('<eos>'))
sentences[-1]

['When you purchase through links on our site, we may earn an affiliate commission.',
 ' Here’s how it works.',
 " If you're in the path of totality, you can contribute to research about the sun.",
 ' Observers who plan to watch the total solar eclipse in April next year can use a new NASA-funded app to record valuable data about the sun, the space agency announced last week.',
 ' Using the free app, SunSketcher 2024, citizen scientists can help advance heliophysics research by clicking as many images of the sun just before and after the solar eclipse.',
 " Those images will help in recording images of a bright, broken ring of sunlight known as Baily's Beads, which shine through mountains and valleys on the edge of the moon just before and after eclipses.",
 ' "There are so many ways to participate in NASA science, especially as we enter the Heliophysics Big Year," space physicist Elizabeth MacDonald, the heliophysics citizen science lead at NASA, said in a statement.',
 ' "We\'re so e

In [13]:
max_tokens_chunk = 500
chunks = []

for i, article_sentences in enumerate(sentences):
  chunks.append([])
  current_chunk = 0
  for sentence in article_sentences:
    if len(chunks[i]) == current_chunk + 1:
      if len(chunks[i][current_chunk]) + len(sentence.split(' ')) <= max_tokens_chunk:
        chunks[i][current_chunk].extend(sentence.split(' '))
      else:
        current_chunk += 1
        chunks[i].append(sentence.split(' '))
    else:
      print(current_chunk)
      chunks[i].append(sentence.split(' '))

0
0
0
0
0
0
0
0
0
0


In [14]:
joined_chunks = []

for i, article_chunks in enumerate(chunks):
    article_text = []

    for chunk_id in range(len(article_chunks)):
        article_text.append(' '.join(article_chunks[chunk_id]))

    joined_chunks.append(article_text)

In [15]:
joined_chunks[0][1]

" Because forward-looking statements relate to matters that have not yet occurred, these statements are inherently subject to risks and uncertainties that could cause our actual results to differ materially from any future results expressed or implied by the forward-looking statements.  Many factors could cause our actual activities or results to differ materially from the activities and results anticipated in forward-looking statements, including, but not limited to, our ability to identify, evaluate and complete any strategic alternative that yields value for our shareholders; the timing and cost of our any pre-clinical or clinical trial, for our product candidates; completion and receiving favorable results of any pre-clinical or clinical trial; regulatory action with respect to Aramchol or any other product candidate by the U. S.  Food and Drug Administration, or the FDA, or the European Medicines Authority, or EMA, including but not limited to acceptance of an application for mark

Summarize all chunks

In [16]:
def summarize_chunks(chunks):
  summarized_texts = []
  for article_chunks in chunks:
    print(len(article_chunks))
    article_text = []
    result = summarizer(article_chunks, max_length=120, min_length=30, do_sample=False)
    article_text = ' '.join([summ['summary_text'] for summ in result])
    summarized_texts.append(article_text)
  return summarized_texts

In [17]:
summarized_texts = summarize_chunks(joined_chunks)

3
3
3
2
3
3
3
4
2
2


In [18]:
summarized_texts[4]

" Japanese astronomer Hideo Nishimura discovered the comet on Aug. 12 . Nishimura's path will keep it close to the sun in the sky as observed from Earth . While it’s definitely bright enough to be visible to the naked eye in dark skies, at best it will hug the horizon just after sunset .  Comet Nishimura is expected to be very active around perihelion (its closest point to the sun) The best chance to see it from Australia comes in the week of Sept. 20 to 27, when the comet's head will set around one hour after the sun . Nishimura will be farthest from the sun in the evening sky on Sept. 23 .  C/2023 A3 (Tsuchinshan-ATLAS) was discovered at the start of this year . Over the next 12 months it will continue to fall sunward, coming closest to the sun in late September 2024 . If it behaves as expected it could be a spectacular sight – but just remember: comets are like cats!"

In [19]:
all_summarized_text = ""
for summarized_text in summarized_texts:
  all_summarized_text += summarized_text

In [20]:
all_summarized_text

' The approval of the patent in the US and the rest of the world is pending . Previously, Galmed reported results showing significant anti-fibrotic effects of Aramchol in a pre-clinical model of lung fibrosis .  Many factors could cause our actual activities or results to differ from the activities and results anticipated in forward-looking statements . The timing and cost of our any pre-clinical or clinical trial, for our product candidates, among other factors, could include the timing, cost or other aspects of the commercial launch of Aramchol or any other product candidate .  You should not rely upon forward-looking statements as predictions of future events . We discuss many of these risks in our Annual Report on Form 20-F for the year ended December 31, 2022 filed with the SEC on March 29, 2023 . Autumn officially begins at 2:50 a.m. EDT on Saturday (Sept. 23) The sun is currently migrating south, having spent the previous six months shining directly on the northern half of our p